In [4]:
pip install -upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib


Usage:   
  /Users/sandesh/opt/anaconda3/bin/python -m pip install [options] <requirement specifier> [package-index-options] ...
  /Users/sandesh/opt/anaconda3/bin/python -m pip install [options] -r <requirements file> [package-index-options] ...
  /Users/sandesh/opt/anaconda3/bin/python -m pip install [options] [-e] <vcs project url> ...
  /Users/sandesh/opt/anaconda3/bin/python -m pip install [options] [-e] <local project path> ...
  /Users/sandesh/opt/anaconda3/bin/python -m pip install [options] <archive url/path> ...

no such option: -u
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [2]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import pickle
import os.path
import base64
import email
from bs4 import BeautifulSoup
  
# Define the SCOPES. If modifying it, delete the token.pickle file.
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']
  
def getEmails():
    # Variable creds will store the user access token.
    # If no valid token found, we will create one.
    creds = None
  
    # The file token.pickle contains the user access token.
    # Check if it exists
    if os.path.exists('token.pickle'):
  
        # Read the token from the file and store it in the variable creds
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
  
    # If credentials are not available or are invalid, ask the user to log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('/Users/sandesh/My_Folder/Python Projects/credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
  
        # Save the access token in token.pickle file for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
  
    # Connect to the Gmail API
    service = build('gmail', 'v1', credentials=creds)
  
    # request a list of all the messages
    result = service.users().messages().list(userId='me').execute()
  
    # We can also pass maxResults to get any number of emails. Like this:
    # result = service.users().messages().list(maxResults=200, userId='me').execute()
    messages = result.get('messages')
  
    # messages is a list of dictionaries where each dictionary contains a message id.
  
    # iterate through all the messages
    for msg in messages:
        # Get the message from its id
        txt = service.users().messages().get(userId='me', id=msg['id']).execute()
  
        # Use try-except to avoid any Errors
        try:
            # Get value of 'payload' from dictionary 'txt'
            payload = txt['payload']
            headers = payload['headers']
  
            # Look for Subject and Sender Email in the headers
            for d in headers:
                if d['name'] == 'Subject':
                    subject = d['value']
                if d['name'] == 'From':
                    sender = d['value']
  
            # The Body of the message is in Encrypted format. So, we have to decode it.
            # Get the data and decode it with base 64 decoder.
            parts = payload.get('parts')[0]
            data = parts['body']['data']
            data = data.replace("-","+").replace("_","/")
            decoded_data = base64.b64decode(data)
  
            # Now, the data obtained is in lxml. So, we will parse 
            # it with BeautifulSoup library
            soup = BeautifulSoup(decoded_data , "lxml")
            body = soup.body()
  
            # Printing the subject, sender's email and message
            print("Subject: ", subject)
            print("From: ", sender)
            print("Message: ", body)
            print('\n')
        except:
            pass
  
  
getEmails()

SyntaxError: invalid non-printable character U+00A0 (2391997136.py, line 9)

In [19]:
pip install html2text

Note: you may need to restart the kernel to use updated packages.


In [31]:
import imaplib
import email
from html2text import html2text
import pandas as pd
import re
import tkinter as tk

# Connect to the server
imap_server = imaplib.IMAP4_SSL('imap.gmail.com')

# Login to the account
imap_server.login('yatharthdoshi5@gmail.com', 'zruzmoahoyewpwwk')

# Select the mailbox you want to read from
imap_server.select('INBOX')


user_input = input("You want to search using subject or sender email: ")

if user_input.upper() == "SUBJECT":
    
    subject_status = input("SEEN or UNSEEN Subject? ")
    subject = input("Enter the subject name: ")
    
    concatenated_string = f'{subject_status.upper()} SUBJECT "{subject}"'
    
elif user_input.upper() == "EMAIL" or user_input.upper() == "SENDER" or user_input.upper() == "SENDER EMAIL" :
    
    sender_email = input("Enter the email address: ")
    
    concatenated_string = f'FROM {sender_email}' 

#status, messages = imap_server.search(None, 'SEEN SUBJECT "abcdefghijklmopqrstuvwxyz"')

status, messages = imap_server.search(None, concatenated_string)

extracted_area = []
extracted_city = []
extracted_state = []
extracted_requirement = []
extracted_email = []
extracted_phone = []
extracted_time = []

# Loop through the messages
for message in messages[0].split():
    # Fetch the message
    status, msg = imap_server.fetch(message, '(RFC822)')

    # Parse the message using the email library
    msg = email.message_from_bytes(msg[0][1])

    # Print the subject and sender of the message
    print(f'Subject: {msg["Subject"]}')
    print(f'From: {msg["From"]}')

    # Print the body of the message
    if msg.is_multipart():
        for part in msg.get_payload():

            
            if part.get_content_type() == "text/html":

                body = html2text(part.get_payload(decode=True).decode(part.get_content_charset()))
               
                user_area = re.findall(r'User Area: (.+)', body)
                user_city = re.findall(r'User City: (.+)', body)
                user_state = re.findall(r'User State: (.+)', body)
                user_requirement = re.findall(r'User Requirement: (.+)', body)
                user_time = re.findall(r'Search Date & Time: (.+)', body)
                user_email = re.findall(r'User Email: (.+)',body)
                user_phone = re.findall(r'User Phone: (.+)', body)
                
                if not user_area and not user_city and not user_state and not user_requirement and (not user_email or not user_phone):
                    pass
                else: 
                    #Extract User Area
                    if user_area: 
                        extracted_area.append(user_area)
                        print(user_area)
                    else: 
                        extracted_area.append("")
                        print("Text not Found!")
                
                    #Extract User City
                    if user_city:                    
                        extracted_city.append(user_city)
                        print(user_city)
                    else: 
                        extracted_city.append("")
                        print("Text not Found!")
                
                    #Extract User State
                    if user_state:                    
                        extracted_state.append(user_state)
                        print(user_state)
                    else: 
                        extracted_state.append("")
                        print("Text not Found!")
                
                    #Extract User Requirement
                    if user_requirement:                    
                        extracted_requirement.append(user_requirement)
                        print(user_requirement)
                    else: 
                        extracted_requirement.append("")
                        print("Text not Found!")
                
                    #Extract User Time
                    if user_time:                    
                        extracted_time.append(user_time)
                        print(user_time)
                    else: 
                        extracted_time.append("")
                        print("Text not Found!")
                    
                    #Extract User Email
                    if user_email:                    
                        extracted_email.append(user_email)
                        print(user_email)
                    else: 
                        extracted_email.append("")
                        print("Text not Found!")
                
                    #Extract User Phone Number
                    if user_phone:                    
                        extracted_phone.append(user_phone)
                        print(user_phone)
                    else: 
                        extracted_phone.append("")
                        print("Text not Found!")
                
    else:
        
        if msg.get_content_type() == "text/html":
            body = html2text(msg.get_payload(decode=True).decode(msg.get_content_charset()))

data = {'Area': extracted_area,'City': extracted_city,'State':extracted_state,'Email':extracted_email,'Phone':extracted_phone,'Requirement': extracted_requirement}
df = pd.DataFrame(data,dtype= str)
df.to_excel('extracted_parts.xlsx', index=False)

# Close the mailbox
imap_server.close()

# Logout from the account
imap_server.logout()

You want to search using subject or sender email: subject
SEEN or UNSEEN Subject? seen
Enter the subject name: Fwd: Enquiry for Onlyweb (Ring Road, Surat)
Subject: Fwd: Enquiry for Onlyweb (Ring Road, Surat)
From: Yatharth Doshi <yatharthdoshi4@gmail.com>
['| Imperial Square Mall-Adajan Road  ']
['| Surat  ']
['| Gujarat  ']
['| Internet Website Designers  ']
['| Wed, 18 Jan 2023 18:38:52  ']
Text not Found!
['| +919265851752  ']
Subject: Fwd: Enquiry for Onlyweb (Ring Road, Surat)
From: Yatharth Doshi <yatharthdoshi4@gmail.com>
['| Vesu  ']
['| Surat  ']
['| Gujarat  ']
['| Computer Graphic Designers  ']
['| Fri, 27 Jan 2023 15:05:48  ']
Text not Found!
['| +918128780520  ']


('BYE', [b'LOGOUT Requested'])

In [1]:
import imaplib
import email
from html2text import html2text
import pandas as pd
import re
import tkinter as tk

def email_reader(concat_string):
    # Connect to the server
    imap_server = imaplib.IMAP4_SSL('imap.gmail.com')

    # Login to the account
    imap_server.login('yatharthdoshi5@gmail.com', 'zruzmoahoyewpwwk')

    # Select the mailbox you want to read from
    imap_server.select('INBOX')
    
    status, messages = imap_server.search(None, concat_string)

    extracted_area = []
    extracted_city = []
    extracted_state = []
    extracted_requirement = []
    extracted_email = []
    extracted_phone = []
    extracted_time = []

    # Loop through the messages
    for message in messages[0].split():
        # Fetch the message
        status, msg = imap_server.fetch(message, '(RFC822)')

        # Parse the message using the email library
        msg = email.message_from_bytes(msg[0][1])

        # Print the subject and sender of the message
        print(f'Subject: {msg["Subject"]}')
        print(f'From: {msg["From"]}')

        # Print the body of the message
        if msg.is_multipart():
            for part in msg.get_payload():

            
                if part.get_content_type() == "text/html":

                    body = html2text(part.get_payload(decode=True).decode(part.get_content_charset()))
               
                    user_area = re.findall(r'User Area: (.+)', body)
                    user_city = re.findall(r'User City: (.+)', body)
                    user_state = re.findall(r'User State: (.+)', body)
                    user_requirement = re.findall(r'User Requirement: (.+)', body)
                    user_time = re.findall(r'Search Date & Time: (.+)', body)
                    user_email = re.findall(r'User Email: (.+)',body)
                    user_phone = re.findall(r'User Phone: (.+)', body)
                
                    if not user_area and not user_city and not user_state and not user_requirement and (not user_email or not user_phone):
                        pass
                    else: 
                        #Extract User Area
                        if user_area: 
                            extracted_area.append(user_area)
                            print(user_area)
                        else: 
                            extracted_area.append("")
                            print("Text not Found!")
                
                        #Extract User City
                        if user_city:                    
                            extracted_city.append(user_city)
                            print(user_city)
                        else: 
                            extracted_city.append("")
                            print("Text not Found!")
                
                        #Extract User State
                        if user_state:                    
                            extracted_state.append(user_state)
                            print(user_state)
                        else: 
                            extracted_state.append("")
                            print("Text not Found!")
                
                        #Extract User Requirement
                        if user_requirement:                    
                            extracted_requirement.append(user_requirement)
                            print(user_requirement)
                        else: 
                            extracted_requirement.append("")
                            print("Text not Found!")
                
                        #Extract User Time
                        if user_time:                    
                            extracted_time.append(user_time)
                            print(user_time)
                        else: 
                            extracted_time.append("")
                            print("Text not Found!")
                    
                        #Extract User Email
                        if user_email:                    
                            extracted_email.append(user_email)
                            print(user_email)
                        else: 
                            extracted_email.append("")
                            print("Text not Found!")
                
                        #Extract User Phone Number
                        if user_phone:                    
                            extracted_phone.append(user_phone)
                            print(user_phone)
                        else: 
                            extracted_phone.append("")
                            print("Text not Found!")
                
        else:
        
            if msg.get_content_type() == "text/html":
                body = html2text(msg.get_payload(decode=True).decode(msg.get_content_charset()))

    data = {'Area': extracted_area,'City': extracted_city,'State':extracted_state,'Email':extracted_email,'Phone':extracted_phone,'Requirement': extracted_requirement}
    df = pd.DataFrame(data,dtype= str)
    df.to_excel('extracted_parts_2.xlsx', index=False)

    # Close the mailbox
    imap_server.close()

    # Logout from the account
    imap_server.logout()

window = tk.Tk()
window.geometry("500x300")
label = tk.Label(window, text="You want to search using subject or sender email: ")
subject_status = tk.Label(window, text="SEEN or UNSEEN Subject? ")
subject = tk.Label(window, text= "Enter the subject name: ")
sender_email = tk.Label(window,text= "Enter the Sender's Email address: ")
error_label = tk.Label(window,text= "Enter valid Criteria!! ",fg="red")

text_area = tk.Entry(window)
ss_text = tk.Entry(window)
subject_text = tk.Entry(window)
sender_email_text = tk.Entry(window)

def email_submit_button():
    
    email = sender_email_text.get()
    
    concatenated_string = f'FROM {email}'
    email_reader(concatenated_string)
    
    success = tk.Label(window, text= "Successfully added your data to excel file",fg="green")
    success.grid(row=6,column=1)
    
    sys.exit()
    

def subject_submit_button():
    
    ss = ss_text.get()
    sub = subject_text.get()
    
    concatenated_string = f'{ss.upper()} SUBJECT "{sub}"'
    print(concatenated_string)  
    email_reader(concatenated_string)
    
    success = tk.Label(window, text= "Successfully added your data to excel file",fg="green")
    success.grid(row=8,column=1)
    
    sys.exit()
     
def on_button_click():
    # Get the text from the Entry widget
    text = text_area.get()
    
    if text.upper() == "SUBJECT": 
        
        subject_status.grid(row=2,column=0)
        ss_text.grid(row=2,column=1)
        
        subject.grid(row=4,column=0)
        subject_text.grid(row=4,column=1)
        
        subject_button.grid(row=6,column=1)
        
    elif text.upper() == "SENDER" or text.upper() == "EMAIL" or text.upper() == "SENDER EMAIL":
        
        sender_email.grid(row=2,column=0)
        sender_email_text.grid(row=2,column=1)
        
        email_button.grid(row=4,column=1)
        
    else: 
        error_label.grid(row=2,column=1)
    
button = tk.Button(window, text="Get Text", command=on_button_click)
subject_button = tk.Button(window, text= "Submit Button", command= subject_submit_button)
email_button = tk.Button(window, text="Submit Button", command= email_submit_button)



label.grid(row=0,column=0)
text_area.grid(row=0,column=1)
button.grid(row=0,column=2)

window.title("Email Scraper")
window.mainloop()

Subject: App
From: Yatharth Doshi <yatharthdoshi4@gmail.com>
Subject: Yatharth
From: Yatharth Doshi <yatharthdoshi4@gmail.com>
Subject: Doshi's
From: Yatharth Doshi <yatharthdoshi4@gmail.com>
Subject: abcdefghijklmopqrstuvwxyz
From: Yatharth Doshi <yatharthdoshi4@gmail.com>
Subject: Fwd: Enquiry for Onlyweb (Ring Road, Surat)
From: Yatharth Doshi <yatharthdoshi4@gmail.com>
['| Imperial Square Mall-Adajan Road  ']
['| Surat  ']
['| Gujarat  ']
['| Internet Website Designers  ']
['| Wed, 18 Jan 2023 18:38:52  ']
Text not Found!
['| +919265851752  ']
Subject: Fwd: Enquiry for Onlyweb (Ring Road, Surat)
From: Yatharth Doshi <yatharthdoshi4@gmail.com>
['| Vesu  ']
['| Surat  ']
['| Gujarat  ']
['| Computer Graphic Designers  ']
['| Fri, 27 Jan 2023 15:05:48  ']
Text not Found!
['| +918128780520  ']
Subject: email
From: Yatharth Doshi <yatharthdoshi4@gmail.com>
Subject: area
From: Yatharth Doshi <yatharthdoshi4@gmail.com>
['Patiya']
['Surat  ']
Text not Found!
Text not Found!
Text not Found!

Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/sandesh/opt/anaconda3/lib/python3.9/tkinter/__init__.py", line 1892, in __call__
    return self.func(*args)
  File "/var/folders/td/8k879ftd3vqg99f58rcggzyr0000gp/T/ipykernel_1592/2188297612.py", line 154, in email_submit_button
    sys.exit()
NameError: name 'sys' is not defined


In [33]:
import tkinter as tk

window = tk.Tk()
window.geometry("500x300")

# Add a label to the window
#label = tk.Label(window, text="Hello, World!")
#label.pack()
label = tk.Label(window, text="Enter your name:")
#label = tk.Label(window, text="Enter your name:", font=("Arial", 14), fg="blue")
text_area = tk.Entry(window)

label.grid(row=0,column=0)
text_area.grid(row=0,column=1)


# Start the main loop
window.mainloop()

In [5]:
import imaplib
import email
from html2text import html2text
import pandas as pd
import re
import tkinter as tk

# Connect to the server
imap_server = imaplib.IMAP4_SSL('imap.gmail.com')

# Login to the account
imap_server.login('yatharthdoshi5@gmail.com', 'zruzmoahoyewpwwk')

# Select the mailbox you want to read from
imap_server.select('INBOX')

window = tk.Tk()
window.geometry("500x300")
bg_image = tk.PhotoImage("/Users/sandesh/My_Folder/scrap.PNG")
label = tk.Label(window, text="You want to search using subject or sender email: ")
subject_status = tk.Label(window, text="SEEN or UNSEEN Subject? ")
subject = tk.Label(window, text= "Enter the subject name: ")

text_area = tk.Entry(window)
ss_text = tk.Entry(window)
subject_text = tk.Entry(window)

def submit_button():
    
    ss = ss_text.get()
    sub = subject_text.get()
    
    concatenated_string = f'{ss.upper()} SUBJECT "{sub}"'
    print(concatenated_string)    
     
def on_button_click():
    # Get the text from the Entry widget
    text = text_area.get()
    
    if text.upper() == "SUBJECT": 
        
        subject_status.grid(row=2,column=0)
        ss_text.grid(row=2,column=1)
        
        subject.grid(row=4,column=0)
        subject_text.grid(row=4,column=1)
        
        button2.grid(row=6,column=1)
    
#        concatenated_string = f'{ss.upper()} SUBJECT "{sub}"'
    
#        print(concatenated_string)
    # Print the text to the console
    #print(text)
    
button = tk.Button(window, text="Get Text", command=on_button_click)
button2 = tk.Button(window, text= "Submit Button", command= submit_button)

#button.pack()
bg_label = tk.Label(window, image=bg_image)
bg_label.place(x=0, y=0, relwidth=1, relheight=1)
label.grid(row=0,column=0)
text_area.grid(row=0,column=1)
button.grid(row=0,column=2)
#canvas.grid(row=0,column=5)
#button2.grid(row=6,column=0)


window.mainloop()